In [ ]:
import cv2
import numpy as np
import scipy as sp
import pandas as pd

import tensorflow as tf
from keras.utils import plot_model

import seaborn as sns
from tqdm import tqdm
from sklearn.preprocessing import MultiLabelBinarizer
import matplotlib.pyplot as plt

tqdm.pandas()
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots

np.random.seed(0)
tf.random.set_seed(0)

import warnings
warnings.filterwarnings("ignore")

In [ ]:
EPOCHS = 20
SAMPLE_LEN = 100
IMAGE_PATH = "../input/plant-pathology-2021-fgvc8/train_images/"
TRAIN_PATH = "../input/plant-pathology-2021-fgvc8/train.csv"

data = pd.read_csv(TRAIN_PATH)
train_data = data.copy()

In [ ]:
def load_image(image_id):
    file_path = image_id
    image = cv2.imread(IMAGE_PATH + file_path)
    return image

train_images = train_data["image"][:SAMPLE_LEN].progress_apply(load_image)

In [ ]:
plt.figure(figsize=(20,12))
labels = sns.barplot(train_data.labels.value_counts().index,train_data.labels.value_counts())
for item in labels.get_xticklabels():
    item.set_rotation(45)

Lượng dữ liệu chênh lệch quá nhiều, đặc biệt dữ liệu đa nhãn, có nhiều bệnh lại chiếm số lượng quá nhỏ. Vì vậy, chúng em đã nghĩ tới cách chỉ giữ lại các nhãn đơn, nhãn kép sẽ đưọc biểu diễn theo nhãn đơn dưới dạng one hot encoding.

# One hot encoding

In [ ]:
train_data['labels'] = train_data['labels'].apply(lambda string: string.split(' '))
s = list(train_data['labels'])
mlb = MultiLabelBinarizer()
trainx = pd.DataFrame(mlb.fit_transform(s), columns=mlb.classes_, index=train_data.index)
trainx

In [ ]:
labels = pd.concat([train_data['image'], trainx], axis=1)
labels.head()
fig = go.Figure([go.Pie(labels=labels.columns[1:],
           values=labels.iloc[:, 1:].sum())])
fig.update_layout(title_text="Pie chart of targets", template="simple_white")
fig.data[0].marker.line.color = 'rgb(0, 0, 0)'
fig.data[0].marker.line.width = 0.5
fig.show()

Dữ liệu sau khi được mã hoá lại đã được phân lại đều hơn, không còn mất cân bằng lớn như trưóc.

# Visualization

In [ ]:
def visual(img):
    fig, ax = plt.subplots(nrows=3, ncols=3, figsize=(30, 20))
    ax[0][0].imshow(load_image(img[0]))
    ax[1][0].imshow(load_image(img[1]))
    ax[2][0].imshow(load_image(img[2]))
    ax[0][1].imshow(load_image(img[3]))
    ax[1][1].imshow(load_image(img[4]))
    ax[2][1].imshow(load_image(img[5]))
    ax[0][2].imshow(load_image(img[6]))
    ax[1][2].imshow(load_image(img[7]))
    ax[2][2].imshow(load_image(img[8]))
    plt.show()

# Healthy

In [ ]:
healthy = data[data['labels']=='healthy']['image'].values[:9]
visual(healthy)

# Scab

In [ ]:
scab = data[data['labels']=='scab']['image'].values[10:19]
visual(scab)

scab và healthy nhìn rất giống nhau, đây có thể là 1 trở ngại rất lớn.

# Frog_eye_leaf_spot

In [ ]:
frog_eye_leaf_spot = data[data['labels']=='frog_eye_leaf_spot']['image'].values[0:9]
visual(frog_eye_leaf_spot)

# Rust

In [ ]:
rust = data[data['labels']=='rust']['image'].values[0:9]
visual(rust)

Chỉ bằng mắt thuờng cũng đã thấy khó phân biệt giữa 2 bệnh rust và frog_eye_leaf_spot. Đây có thể là khó khăn trong quá trình đào tạo.

# Complex

In [ ]:
complex = data[data['labels']=='complex']['image'].values[0:9]
visual(complex)

# Powdery_mildew

In [ ]:
powdery_mildew = data[data['labels']=='powdery_mildew']['image'].values[0:9]
visual(powdery_mildew)

In [ ]:
def visualize_leaves(cond=[0, 0, 0, 0, 0, 0], cond_cols=["healthy"], is_cond=True):
    if not is_cond:
        cols, rows = 3, min([3, len(train_images)//3])
        fig, ax = plt.subplots(nrows=rows, ncols=cols, figsize=(30, rows*20/3))
        for col in range(cols):
            for row in range(rows):
                ax[row, col].imshow(train_images.loc[train_images.index[-row*3-col-1]])
        return None
        
    cond_0 = "complex == {}".format(cond[0])
    cond_1 = "frog_eye_leaf_spot == {}".format(cond[1])
    cond_2 = "healthy == {}".format(cond[2])
    cond_3 = "powdery_mildew == {}".format(cond[3])
    cond_4 = "rust == {}".format(cond[4])
    cond_5 = "scab == {}".format(cond[5])
    cond_list = []
    for col in cond_cols:
        if col == "complex":
            cond_list.append(cond_0)
        if col == "frog_eye_leaf_spot":
            cond_list.append(cond_1)
        if col == "healthy":
            cond_list.append(cond_2)
        if col == "powdery_mildew":
            cond_list.append(cond_3)
        if col == "rust":
            cond_list.append(cond_4)
        if col == "scab":
            cond_list.append(cond_5)
    
    data = labels.loc[:100]
    for cond in cond_list:
        data = data.query(cond)
        
    images = train_images.loc[list(data.index)]
    cols, rows = 3, min([3, len(images)//3])
    
    fig, ax = plt.subplots(nrows=rows, ncols=cols, figsize=(30, rows*20/3))
    for col in range(cols):
        for row in range(rows):
            ax[row, col].imshow(images.loc[images.index[row*3+col]])
    plt.show()

In [ ]:
fig = px.parallel_categories(labels[['complex', 'frog_eye_leaf_spot', 'healthy', 'powdery_mildew', 'rust','scab']], color="healthy", color_continuous_scale="sunset",\
                             title="Parallel categories plot of targets")
fig

Đồ thị trên cho ta thấy sự liên quan giữa các nhãn với nhau. Ví dụ với các lá khỏe mạnh, đường màu xanh sẽ chỉ đi qua "healthy" có giá trị = 1 còn các nhãn khác đều đi qua vùng có giá trị = 0. Độ rộng của đường trên các khoảng thể hiện tần suất xuất hiện của nhãn đó khi kết hợp với các nhãn khác.

# Augmentation

# Histogram Equalization

In [ ]:
def histogram(img):
    R, G, B = cv2.split(img)

    output1_R = cv2.equalizeHist(R)
    output1_G = cv2.equalizeHist(G)
    output1_B = cv2.equalizeHist(B)

    equ = cv2.merge((output1_R, output1_G, output1_B))
    fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(30, 20))
    ax[0].imshow(img)
    ax[0].set_title('Original Image', fontsize=24)
    ax[1].imshow(equ)
    ax[1].set_title('Histogram Equalization Image', fontsize=24)
    plt.show()

In [ ]:
histogram(train_images[9])
histogram(train_images[10])
histogram(train_images[11])

Sau khi cân bằng histogram, ta thấy các vị trí bị bệnh trên chiếc lá sẽ nổi bật hẳn trên nền của lá. Nên ta sẽ thêm lượng dữ liệu được cân bằng histogram này nhằm giúp nhấn mạnh vùng bị bệnh.

# Canny Detection, Crop image (Tìm đối tượng lá trong ảnh)

In [ ]:
def edge_and_cut(img):
    emb_img = img.copy()
    edges = cv2.Canny(img, 100, 200)
    edge_coors = []
    for i in range(edges.shape[0]):
        for j in range(edges.shape[1]):
            if edges[i][j] != 0:
                edge_coors.append((i, j))
    
    row_min = edge_coors[np.argsort([coor[0] for coor in edge_coors])[0]][0]
    row_max = edge_coors[np.argsort([coor[0] for coor in edge_coors])[-1]][0]
    col_min = edge_coors[np.argsort([coor[1] for coor in edge_coors])[0]][1]
    col_max = edge_coors[np.argsort([coor[1] for coor in edge_coors])[-1]][1]
    new_img = img[row_min:row_max, col_min:col_max]
    
    emb_img[row_min-10:row_min+10, col_min:col_max] = [255, 0, 0]
    emb_img[row_max-10:row_max+10, col_min:col_max] = [255, 0, 0]
    emb_img[row_min:row_max, col_min-10:col_min+10] = [255, 0, 0]
    emb_img[row_min:row_max, col_max-10:col_max+10] = [255, 0, 0]
    
    fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(30, 20))
    ax[0].imshow(img, cmap='gray')
    ax[0].set_title('Original Image', fontsize=24)
    ax[1].imshow(edges, cmap='gray')
    ax[1].set_title('Canny Edges', fontsize=24)
    ax[2].imshow(emb_img, cmap='gray')
    ax[2].set_title('Bounding Box', fontsize=24)
    plt.show()

In [ ]:
edge_and_cut(train_images[3])
edge_and_cut(train_images[4])
edge_and_cut(train_images[5])

Áp dụng Canny Detection để tìm cạnh của lá, sau đó ở tập dữ liệu cạnh, tìm X_max, Y_max, X_min, Y_min là 4 toạ độ của vùng chữ nhật chứa chiếc lá. Sau đấy ta áp dụng để Crop ra vùng chứa chiếc lá đó để tạo thêm ảnh (có ít background) giúp gia tăng bộ dữ liệu. Ta áp dụng được cách này vì chiếc lá là đối tượng chính và rõ nét nhất.

# Rotate

In [ ]:
def rotate(img):
    fig, ax = plt.subplots(nrows=1, ncols=4, figsize=(30, 20))
    ax[0].imshow(img)
    ax[0].set_title('Original Image', fontsize=24)
    ax[1].imshow(cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE))
    ax[1].set_title('90', fontsize=24)
    ax[2].imshow(cv2.rotate(img, cv2.ROTATE_90_COUNTERCLOCKWISE))
    ax[2].set_title('180', fontsize=24)
    ax[3].imshow(cv2.rotate(img, cv2.ROTATE_180))
    ax[3].set_title('270', fontsize=24)
    plt.show()

In [ ]:
rotate(train_images[9])
rotate(train_images[10])

Vì đối tượng trong ảnh là lá, quay chiều nào cũng mang lại hình ảnh chiếc lá nên ta sẽ áp dụng cách này để tăng bộ dữ liệu.

# Flip

In [ ]:
def flip(img):
    fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(30, 20))
    ax[0].imshow(img)
    ax[0].set_title('Original Image', fontsize=24)
    ax[1].imshow(cv2.flip(img, 0))
    ax[1].set_title('Vertical Flip', fontsize=24)
    ax[2].imshow(cv2.flip(img, 1))
    ax[2].set_title('Horizontal Flip', fontsize=24)
    plt.show()

In [ ]:
flip(train_images[9])
flip(train_images[10])
flip(train_images[11])

Vì đối tượng trong ảnh là lá, lật chiều nào cũng mang lại hình ảnh chiếc lá nên ta sẽ áp dụng cách này để tăng bộ dữ liệu.